# How to work with Points

The types `Point{D}` (an efficient immutable point) and `MPoint{D}` (a flexible mutable point) are implemented as follows:

In [ ]:
using StaticArrays

In [ ]:
const Point{D} = SVector{D,Float64} where D
const MPoint{D} = MVector{D,Float64} where D

Since the parameter `D` should be a constant, you can use dispatch on so-called [value types](https://docs.julialang.org/en/v1/manual/types/#%22Value-types%22-1) in functions that work with points and have no other mean to figure out what `D` is. E.g., if we want a function that allocates a 1D array of points of length `n`, we can procede as follows

In [ ]:
function allocate_points(n,::Val{D}) where D
    Array{Point{D},1}(undef,(n,))
end

And we use it as follows to create a long array of `Point{3}`. Recall that `Val(3)` is the only instance of the type `Val{3}`.

In [ ]:
@time a = allocate_points(10000000,Val(3))

`Point{D}` is immutable, what means that you can only create or assign a whole point at once. However, in some situations it is needed to set or modify a point component by component. In this case we can use `MPoint{D}`a mutable version of `Point{D}`. E.g., in the following function, we allocate a single `MPoint{D}` to fill an array of immutable `Point{D}`.

In [ ]:
function fill_points!(a::Array{Point{D},1}) where D
    p = MPoint{D}(zeros(D))
    @inbounds for i in 1:length(a)
        @inbounds for j in 1:D
            p[j] = i*j # Here we take advantatge of the mutability of p
        end
        a[i] = p # Here we assign a whole immutable point at once
    end
    a
end

In [ ]:
@time fill_points!(a)

`Point{D}` isntances are immutable, so they are allocated in the stack. For this reason it is efficient to create and return a `Point{D}` from inside a function that is called multiple times.

In [ ]:
function inner_fun(::Val{D}) where D
    if D == 3
        qx = 1.0
        qy = 1.4
        qz = 2.0
        q = Point{D}(qx,qy,qz)
    else
        q = @SVector ones(Float64,D)
    end
    q
end

function outer_fun!(a::Array{Point{D},1})  where D
    @inbounds for i=1:length(a)
      a[i] = inner_fun(Val(D))
    end
    a
end

In [ ]:
@time outer_fun!(a)